In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm as tqdm #進度條

train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print('CUDA is available!  Training on GPU ...')
else:
    print('CUDA is not available.  Training on CPU ...')

CUDA is available!  Training on GPU ...


In [2]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        x = torch.tensor(self.data[index][:-1], dtype=torch.float32)
        y = torch.tensor(self.data[index][-1], dtype=torch.float32)
        return x, y

    def __len__(self):
        return len(self.data)

In [3]:
class Willy_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 128)
        self.layer2 = nn.Linear(128, 1)
    def forward(self, x):
        out = x.view(-1, 2)
        out = F.relu(self.layer1(out))
        out = self.layer2(out)
        return out

In [4]:
# hyperparameter
epoch = 10
willy_model = Willy_Model()

# read data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

train = train_df[['x1','x2','y']].to_numpy()
test = test_df[['x1','x2']].to_numpy()

train_loader = DataLoader(dataset=MyDataset(train), batch_size=64, shuffle=True, num_workers=0)

In [7]:
def model_train(model, train_loader, test_loader = None, epochs = 5, save_name='willy_model', LR = 0.001, EPS = 1e-7):
    if train_on_gpu:
        model.cuda()
        
    history = {'accuracy':[] ,'val_accuracy':[] ,'loss':[] ,'val_loss':[]}
    # Learning Rate
    LR = LR
    EPS = EPS
    valid_loss_min = np.Inf # track change in validation loss
    train_loss_min = np.Inf

    optimizer = torch.optim.Adam(model.parameters(), lr=LR ,eps=EPS)  # optimize all cnn parameters
    criterion = nn.MSELoss()

    for epoch in range(1, epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        print('running epoch: {}'.format(epoch))
        ###################
        # train the model #
        ###################
        model.train()
        for data, target in tqdm(train_loader):
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.view(-1,1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        train_loss = train_loss/len(train_loader.dataset)
        
        ######################    
        # validate the model #
        ######################
        if test_loader != None: #避免沒有testing(validation) data
            model.eval()
            with torch.no_grad():
                for data, target in tqdm(test_loader):
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()
                    output = model(data)
                    loss = criterion(output, target)
                    valid_loss += loss.item()*data.size(0)
            valid_loss = valid_loss/len(test_loader.dataset)

        
        history['loss'].append(train_loss)
        history['val_loss'].append(valid_loss)
        # print training/validation statistics 
        print('Training Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            train_loss, valid_loss))

        # save model if validation loss has decreased
        if train_loss <= train_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(train_loss_min, train_loss))
            torch.save(model.state_dict(), '{}.pth'.format(save_name))
            train_loss_min = train_loss
        if epoch % 10 == 0:
            torch.save(model.state_dict(), '{}_{}.pth'.format(save_name ,(str(epoch + 10))))
    return history

In [8]:
# willy_model.load_state_dict(torch.load('willy_model.pth'))
model_train(willy_model, train_loader , epochs = 500)

running epoch: 1


  0%|          | 0/125 [00:00<?, ?it/s]

Training Loss: 0.138055 	Validation Loss: 0.000000
Validation loss decreased (inf --> 0.138055).  Saving model ...
running epoch: 2


  0%|          | 0/125 [00:00<?, ?it/s]

Training Loss: 0.120084 	Validation Loss: 0.000000
Validation loss decreased (0.138055 --> 0.120084).  Saving model ...
running epoch: 3


...

  0%|          | 0/125 [00:00<?, ?it/s]

Training Loss: 0.025720 	Validation Loss: 0.000000
running epoch: 500


  0%|          | 0/125 [00:00<?, ?it/s]

Training Loss: 0.025071 	Validation Loss: 0.000000


{'accuracy': [],
 'val_accuracy': [],
 'loss': [0.13805511248111724,
  0.12008372217416763,
  0.11310478711128234,
  0.10964776647090912,
  0.10614538514614105,
  0.10290748935937881,
  0.10093061703443527,
  0.09864743649959565,
  0.09626607343554497,
  0.0945667153596878,
  0.09304277354478836,
  0.09075180020928383,
  0.08918208265304566,
  0.0877506816983223,
  0.08618218180537224,
  0.08458212915062904,
  0.08318784135580062,
  0.08151555994153023,
  0.08065316298604011,
  0.07961342576146126,
  0.07838223850727082,
  0.07758068254590035,
  0.07657793134450913,
  0.07519629460573196,
  0.07414439845085144,
  0.07343177095055581,
  0.07270059499144554,
  0.07167514848709107,
  0.07057349920272828,
  0.0700443671643734,
  0.06942284786701203,
  0.06843504267930985,
  0.06774494555592538,
  0.0668133972287178,
  0.06660486486554146,
  0.06524609997868538,
  0.06506132581830025,
  0.06435600465536118,
  0.06370644068717957,
  0.06278399360179901,
  0.06229100835323334,
  0.06237792721

In [6]:
# willy_model.load_state_dict(torch.load('willy_model.pth'))
willy_model.eval()

start = 0
interval = 100
end = start + interval

test_input = train[start:end,:2]
test_ans = train[start:end,2]
with torch.no_grad():
    test_output = willy_model(torch.tensor(test_input, dtype=torch.float, device='cuda:0')).cpu().numpy().reshape(-1)
    print(test_output)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [80]:
print(test_ans)
print(test_output)

[0.659 1.37  1.37  0.726 1.3   0.888 1.02  1.4   0.76  1.57  0.489 0.965
 1.5   1.15  1.49  0.637 0.514 1.34  1.21  0.861 0.91  0.983 0.606 0.706
 0.655 0.523 1.71  1.5   0.807 1.44  1.43  0.673 0.544 1.02  0.563 1.12
 0.849 0.847 1.02  0.558 1.25  1.58  0.917 1.04  1.24  0.653 1.13  1.17
 1.58  0.854 1.26  0.985 1.06  1.54  1.51  0.831 1.18  1.25  1.12  1.21
 0.675 0.482 1.29  0.84  1.34  1.55  1.13  0.536 1.34  0.301 1.12  0.525
 1.36  1.59  0.416 0.831 1.14  1.41  1.54  1.33  0.553 1.48  1.55  0.905
 1.43  1.05  1.19  1.16  1.63  1.42  1.52  1.37  0.98  0.792 1.11  1.1
 1.33  1.22  0.56  1.12 ]
[0.57853335 1.1450839  1.5481336  0.7528571  1.3920121  0.6448078
 0.94644886 1.4164798  0.88569075 1.405441   0.47888333 0.9726191
 1.4017484  1.1229296  1.4997175  0.9143984  0.46845052 1.2432897
 1.0574832  0.90003794 0.6663917  1.1073239  0.54613215 0.6508171
 0.5692049  0.5027049  1.4208765  1.3446774  0.4761843  1.5130033
 1.4778764  0.75907946 0.512518   1.0920925  0.7350532  1.3487766

In [81]:
MSE = (test_ans-test_output)**2

In [82]:
MSE.sum()/interval

0.02107256729249299